In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from header import *
import psutil
from mne.stats import spatio_temporal_cluster_1samp_test, spatio_temporal_cluster_test, permutation_cluster_1samp_test, permutation_cluster_test, summarize_clusters_stc
from scipy.stats.distributions import f,t
from tqdm import tqdm
import visbrain
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
t0 = time.perf_counter()
task = 'SMEG' #'MIMOSA'
states = ['RS','FA','OM']
subjects = get_subjlist(task)
subjects = subjects[:subjects.index('109')]

reject = ['069', '072', '030', '054', '071', '090', '093']
for sub in reject:
    if sub in subjects:
        subjects.remove(sub)

subjects.sort()
#subjects = ['002', '007', '012', '014', '018', '032', '037', '042', '083', '087', '089', '091', '050', '055', '064', '067', '068', '070', '073']
experts = []
novices = []
experts_i = []
novices_i = []
for s,sub in enumerate(subjects):
    if expertise(sub) == 'N':
        novices.append(sub)
        novices_i.append(s)
    if expertise(sub) == 'E':
        experts.append(sub)
        experts_i.append(s)

In [3]:
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[-1]
noise_cov = 'empty_room_cov'
stc_ext = '-lh.stc'
sfreq = 200
surface = 'ico4'
start = None
stop = None
stc_path = op.join(Analysis_path, task, 'meg', 'SourceEstimate')
evo_path = op.join(Analysis_path, task, 'meg', 'Evoked')

# Evoked

In [4]:
data_file = op.join(evo_path, 'DATASET-{}Hz.nc'.format(sfreq))
evo = xr.open_dataarray(data_file, name)
evo.load()

<xarray.DataArray (state: 3, subject: 34, time: 90, sensor: 275)>
array([[[[ -7.273666e-16, ...,  -5.160424e-15],
         ..., 
         [ -4.482074e-15, ...,  -4.317540e-16]],

        ..., 
        [[  2.196487e-15, ...,   1.408550e-15],
         ..., 
         [  5.216744e-15, ...,   1.765455e-14]]],


       ..., 
       [[[  2.321139e-15, ...,  -1.001085e-15],
         ..., 
         [ -5.865644e-15, ...,   7.151555e-15]],

        ..., 
        [[  1.841743e-14, ...,  -1.249284e-14],
         ..., 
         [  1.584615e-14, ...,  -1.187543e-14]]]])
Coordinates:
  * time     (time) float64 -0.175 -0.17 -0.165 -0.16 -0.155 -0.15 -0.145 ...
  * subject  (subject) object '002' '004' '007' '010' '012' '014' '016' ...
  * state    (state) object 'RS' 'FA' 'OM'
  * sensor   (sensor) object 'MLC11' 'MLC12' 'MLC13' 'MLC14' 'MLC15' 'MLC16' ...

In [5]:
def evoked_perm_test(X1, X2, stat_file, test_key, times, sensors, mode='a', p_threshold=0.01, connectivity=None, paired=False, fif_significance=0.05):
    """
    If paired, test X1-X2.
    A summary SourceEstimate of the stats is saved if there is a significant cluster (p-value < fif_significance).
    Saving can be forced by setting fif_significance to 1, or disabled by setting it to 0.
    Input: arrays of shape (subjects, time, space)
    """
    os.makedirs(op.dirname(stat_file), exist_ok=True)
    evoked_file = op.splitext(stat_file)[0] + '_' + test_key + '-stat.fif'
    
    if not isinstance(X2, (np.ndarray, xr.DataArray, list)):
        X2 = np.zeros(X1.shape)
    
    if paired:
        X = X1 - X2
        t_threshold = -t.ppf(p_threshold / 2, X.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_1samp_test(X, connectivity=connectivity, threshold=t_threshold, n_jobs=4)
    else:
        f_threshold = f.ppf(1 - p_threshold / 2, X1.shape[0] - 1, X2.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_test([X1,X2], connectivity=connectivity, threshold=f_threshold, n_jobs=4)
    
    p_val = np.ones_like(T_obs)
    clu_inds = np.zeros_like(T_obs)
    info = mne.create_info(sensors, np.round(1 / (times[1] - times[0])), 'mag')
    
    for c,clu in enumerate(clusters):
        p_val[clu] = cluster_pv[c]
        clu_inds[clu] = c+1
        if np.where(cluster_pv[c] <= fif_significance)[0].size:
            data = np.full_like(T_obs, np.nan)
            data[clu] = T_obs[clu]
            mne.write_evokeds(evoked_file, mne.EvokedArray(data.T, info, times[0], 'cluster_{}'.format(c+1)))
    
    if np.where(p_val <= fif_significance)[0].size:
        mne.write_evokeds(evoked_file, mne.EvokedArray(np.where(p_val <= fif_significance, T_obs, np.nan).T, info, times[0], 'all_clusters'))
    
    stats = xr.DataArray(np.zeros((3, *T_obs.shape)), dims=['data', 'time', 'sensor'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'time':times, 'sensor':sensors})
    stats.loc['T_stat'] = T_obs
    stats.loc['p_val'] = p_val
    stats.loc['clu_inds'] = clu_inds
    
    stats.to_netcdf(path=stat_file, group=test_key, mode=mode if op.isfile(stat_file) else 'w')
    return clu_all

In [6]:
start = .1#evo.time.values[0]
stop = .25#evo.time.values[-1]
stat_path = op.join(Analysis_path, task, 'meg', 'Stats', 'evoked_{}_{}'.format(start, stop))
os.makedirs(stat_path, exist_ok=True)
stat_file = op.join(stat_path, '{}-{}Hz.nc'.format(name, sfreq))
paired_tests = {'RS_vs_FA':('RS', 'FA', subjects), 'RS_vs_OM':('RS', 'OM', subjects),
         'OM_vs_FA':('OM', 'FA', subjects), 'RS_vs_FA+E':('RS', 'FA', experts),
         'RS_vs_OM+E':('RS', 'OM', experts), 'OM_vs_FA+E':('OM', 'FA', experts),
         'RS_vs_FA+N':('RS', 'FA', novices), 'RS_vs_OM+N':('RS', 'OM', novices),
         'OM_vs_FA+N':('OM', 'FA', novices)}
clu = dict()

In [11]:
for key,val in paired_tests.items():
    clu[key] = evoked_perm_test(evo.loc[val[0],val[2],start:stop].values, evo.loc[val[1],val[2],start:stop].values, stat_file=stat_file, test_key=key, times=evo.loc[:,:,start:stop].time.values, sensors=evo.sensor.values.tolist(), paired=True)

stat_fun(H1): min=-3.903630 max=4.331629
Running initial clustering
Found 48 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.3s remaining:    3.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    5.5s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-4.195056 max=3.370257
Running initial clustering
Found 27 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-3.504096 max=3.761773
Running initial clustering
Found 47 clusters
Permuting 1023 times...


...

Computing cluster p-values
Done.
stat_fun(H1): min=-3.548634 max=4.546885
Running initial clustering
Found 22 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    3.2s remaining:    3.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.7s finished


Computing cluster p-values
Done.
stat_fun(H1): min=-3.471529 max=4.742145
Running initial clustering
Found 47 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    4.9s finished


Computing cluster p-values
Done.


In [10]:
evo.time.values

array([-0.175, -0.17 , -0.165, -0.16 , -0.155, -0.15 , -0.145, -0.14 ,
       -0.135, -0.13 , -0.125, -0.12 , -0.115, -0.11 , -0.105, -0.1  ,
       -0.095, -0.09 , -0.085, -0.08 , -0.075, -0.07 , -0.065, -0.06 ,
       -0.055, -0.05 , -0.045, -0.04 , -0.035, -0.03 , -0.025, -0.02 ,
       -0.015, -0.01 , -0.005,  0.   ,  0.005,  0.01 ,  0.015,  0.02 ,
        0.025,  0.03 ,  0.035,  0.04 ,  0.045,  0.05 ,  0.055,  0.06 ,
        0.065,  0.07 ,  0.075,  0.08 ,  0.085,  0.09 ,  0.095,  0.1  ,
        0.105,  0.11 ,  0.115,  0.12 ,  0.125,  0.13 ,  0.135,  0.14 ,
        0.145,  0.15 ,  0.155,  0.16 ,  0.165,  0.17 ,  0.175,  0.18 ,
        0.185,  0.19 ,  0.195,  0.2  ,  0.205,  0.21 ,  0.215,  0.22 ,
        0.225,  0.23 ,  0.235,  0.24 ,  0.245,  0.25 ,  0.255,  0.26 ,
        0.265,  0.27 ])

# SourceEstimate

In [6]:
data_file = op.join(stc_path, 'fsaverage', 'DATASET-{}-surface_{}-{}Hz-beamformer.nc'.format(noise_cov, surface, sfreq))
stc = xr.open_dataarray(data_file, name)
stc.load()

<xarray.DataArray (state: 3, subject: 34, time: 50, src: 20484)>
array([[[[ 8.556109e-03, ...,  2.134121e-02],
         ...,
         [ 4.530383e-02, ...,  1.810445e-02]],

        ...,

        [[ 1.169846e-01, ...,  8.895773e-02],
         ...,
         [ 1.032996e-01, ...,  1.014239e-01]]],


       ...,


       [[[ 3.722164e-02, ...,  3.015478e-02],
         ...,
         [ 2.718116e-02, ...,  2.640520e-02]],

        ...,

        [[-4.905814e-06, ...,  5.309783e-02],
         ...,
         [ 9.928549e-02, ...,  2.555502e-02]]]])
Coordinates:
  * time     (time) float64 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 ...
  * subject  (subject) object '002' '004' '007' '010' '012' '014' '016' ...
  * state    (state) object 'RS' 'FA' 'OM'
  * src      (src) object 'lh_0' 'lh_1' 'lh_2' 'lh_3' 'lh_4' 'lh_5' 'lh_6' ...

In [17]:
def cluster_perm_test(X1, X2, stat_file, test_key, times, vertices, mode='a', p_threshold=0.01, connectivity='ico5', paired=False, fif_significance=0.05, tmin=0):
    """
    If paired, test X1-X2.
    A summary SourceEstimate of the stats is saved if there is a significant cluster (p-value < fif_significance).
    Saving can be forced by setting fif_significance to 1, or disabled by setting it to 0.
    Input: arrays of shape (subjects, time, space)
    """
    if not isinstance(X2, (np.ndarray, xr.DataArray)):
        X2 = np.zeros(X1.shape)
    
    if type(connectivity) is str:
        connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(connectivity[-1])))
    
    if paired:
        X = X1 - X2
        t_threshold = -t.ppf(p_threshold / 2, X.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_1samp_test(X, connectivity=connectivity, threshold=t_threshold, n_jobs=4)
    else:
        f_threshold = f.ppf(1 - p_threshold / 2, X1.shape[0] - 1, X2.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = clu_all = spatio_temporal_cluster_test([X1,X2], connectivity=connectivity, threshold=f_threshold, n_jobs=4)
    
    p_val = np.ones(T_obs.shape)
    clu_inds = np.zeros(T_obs.shape)
    
    for c,clu in enumerate(clusters):
        p_val[clu] = cluster_pv[c]
        clu_inds[clu] = c+1
    
    stats = xr.DataArray(np.zeros((3, 2, len(times), len(vertices))), dims=['data', 'hemisphere', 'time', 'src'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'hemisphere':['left', 'right'], 'time':times, 'src':vertices})
    for h,hemi in enumerate(['left', 'right']):
        i = h + 1
        h *= len(vertices)
        i *= len(vertices)
        stats.loc['T_stat', hemi] = T_obs[:, h:i]
        stats.loc['p_val', hemi] = p_val[:, h:i]
        stats.loc['clu_inds', hemi] = clu_inds[:, h:i]
    
    if not op.isdir(op.split(stat_file)[0]):
        os.makedirs(op.split(stat_file)[0])
    
    if not op.isfile(stat_file):
        mode = 'w'
    
    stats.to_netcdf(path=stat_file, group=test_key, mode=mode)
    
    if np.where(cluster_pv <= fif_significance)[0].size:
        clu_sum = summarize_clusters_stc(clu_all, tstep=1, tmin=0)
        clu_sum.save(op.splitext(stat_file)[0] + '-' + test_key + '-summary')
        
        stats_stc = mne.SourceEstimate(np.where(p_val <= fif_significance, T_obs, 0).T, [np.arange(10242), np.arange(10242)], times[0], 1/sfreq, 'fsaverage')
        stats_stc.save(op.splitext(stat_file)[0] + '-' + test_key)
    
    return clu_all

In [8]:
spacing='ico5'
connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(spacing[-1])))

-- number of connected vertices : 20484


In [9]:
start = stc.time.values[0]
stop = stc.time.values[-1]
vertno = np.arange(stc.shape[-1]/2, dtype=int)

stat_path = op.join(Analysis_path, task, 'meg', 'Stats', '{}_{}'.format(round(start,4),round(stop,4)))
os.makedirs(stat_path, exist_ok=True)
stat_file = op.join(stat_path, '{}-{}-surface_{}-{}Hz-beamformer.nc'.format(name, noise_cov, surface, sfreq))

paired_tests = {'RS_vs_FA':('RS', 'FA', subjects), 'RS_vs_OM':('RS', 'OM', subjects),
         'OM_vs_FA':('OM', 'FA', subjects), 'RS_vs_FA+E':('RS', 'FA', experts),
         'RS_vs_OM+E':('RS', 'OM', experts), 'OM_vs_FA+E':('OM', 'FA', experts),
         'RS_vs_FA+N':('RS', 'FA', novices), 'RS_vs_OM+N':('RS', 'OM', novices),
         'OM_vs_FA+N':('OM', 'FA', novices)}
clu = dict()

# Heartbeat Evoked Response

In [16]:
clu['RS_vs_FA'][0].shape

(50, 20484)

In [35]:
test_key = 'HER_0.0001'
#X1 = stc.values.reshape((np.multiply(*stc.shape[0:2]),*stc.shape[2:]))
X1 = stc.values.mean(axis=0)
X2 = np.zeros(X1.shape)
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(X1, X2, stat_file=stat_file, test_key=test_key, times=stc.time.values, vertices=vertno, connectivity=connectivity, paired=True, p_threshold=0.0001)

stat_fun(H1): min=2.469534 max=41.924578
Running initial clustering
Found 6 clusters
Permuting 1023 times...
[...................................     ] 87.84314 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.2min remaining:  1.2min


[....................................... ] 99.22481 |    Computing cluster p-values
Done.


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.3min finished


In [31]:
X1 = stc.values.mean(axis=0)

In [39]:
X1.shape

(34, 30, 20484)

# States tests

In [18]:
for key,val in paired_tests.items():
    clu[key] = cluster_perm_test(stc.loc[val[0],val[2],start:stop].values, stc.loc[val[1],val[2],start:stop].values, stat_file=stat_file, test_key=key, times=stc.loc[:,:,start:stop].time.values, vertices=vertno, connectivity=connectivity, paired=True)

stat_fun(H1): min=-5.528916 max=3.513895
Running initial clustering
Found 489 clusters
Permuting 1023 times...
[..................................      ] 86.82171 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  4.0min remaining:  4.0min


[....................................... ] 99.22481 |    Computing cluster p-values


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.2min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.2min finished


Done.
Writing STC to disk...
[done]
Writing STC to disk...
[done]
stat_fun(H1): min=-6.958375 max=4.619963
Running initial clustering
Found 389 clusters
Permuting 1023 times...
[..................................      ] 86.82171 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  3.8min remaining:  3.8min


[....................................... ] 99.22481 |    

[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.0min finished


Computing cluster p-values
Done.
Writing STC to disk...
[done]
Writing STC to disk...
[done]


In [14]:
t0 = time.perf_counter()
ti = t0
test_key = 'RS_vs_FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',:,start:stop].values, stc.loc['FA',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',:,start:stop].values, stc.loc['OM',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['OM',:,start:stop].values, stc.loc['FA',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_FA+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',experts,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',experts,start:stop].values, stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['OM',experts,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_FA+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['FA',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['OM',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values, stc.loc['FA',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()


test_key = 'N_vs_E+RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values, stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA-RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values-stc.loc['RS',novices,start:stop].values, stc.loc['FA',experts,start:stop].values-stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+OM-RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values-stc.loc['RS',novices,start:stop].values, stc.loc['OM',experts,start:stop].values-stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA-OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values-stc.loc['OM',novices,start:stop].values, stc.loc['FA',experts,start:stop].values-stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()


T = ti - t0
print('\n*****\nTotal running time:', T)

stat_fun(H1): min=-4.626214 max=3.502408
Running initial clustering
Found 234 clusters
Permuting 1023 times...
[..................................      ] 86.82171 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.4min remaining:  2.4min


[....................................... ] 99.22481 |    Computing cluster p-values


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.5min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.5min finished


Done.
	 RS_vs_FA 169.63150782399862 
	*****

stat_fun(H1): min=-5.680200 max=4.164851
Running initial clustering
Found 296 clusters
Permuting 1023 times...
[..................................      ] 86.82171 \    

...

	 N_vs_E+OM-RS 521.5901186310002 
	*****

stat_fun(H1): min=0.000000 max=29.058168
Running initial clustering
Found 392 clusters
Permuting 1023 times...
[....................................... ] 99.22481 |    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  8.4min remaining:  8.4min


Computing cluster p-values
Done.


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  8.4min finished


	 N_vs_E+FA-OM 509.4705776610008 
	*****


*****
Total running time: 4437.454123295


In [3]:
names = ['T_ECG_included', 'T_ECG_excluded']#'R_ECG_included', 'R_ECG_excluded', 
name = names[0]
noise_cov = 'empty_room_cov'
sfreq = 200
win = (-.175,-.075)#(.1,.2)
surface = 'ico4'

stc = dict()
for i,w in enumerate([(-.175,-.075),(.1,.2)]):
    data_file = op.join(Analysis_path, task, 'meg', 'SourceEstimate', 'fsaverage', 'DATASET-{}-surface_{}-{}_{}-{}Hz.nc'.format(noise_cov, surface, *w, sfreq))
    stc[i] = xr.open_dataarray(data_file, name)
    stc[i].load()

In [4]:
T_obs=dict(); clusters=dict(); cluster_pv=dict(); H0=dict(); clu=dict()
vertno = np.arange(stc[0].shape[-1]/2, dtype=int)

stat_path = op.join(Analysis_path, task, 'meg', 'Stats', 'baseline')
os.makedirs(stat_path, exist_ok=True)
stat_file = op.join(stat_path, '{}-{}-surface_{}-{}Hz-baseline.nc'.format(name, noise_cov, surface, sfreq))

In [ ]:
test_key = 'RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc[1].loc['RS'].values, stc[0].loc['RS'].values, stat_file=stat_file, test_key=test_key, times=stc[0].time, vertices=vertno, connectivity=connectivity, paired=True)
test_key = 'FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc[1].loc['FA'].values, stc[0].loc['FA'].values, stat_file=stat_file, test_key=test_key, times=stc[0].time, vertices=vertno, connectivity=connectivity, paired=True)
test_key = 'OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test(stc[1].loc['OM'].values, stc[0].loc['OM'].values, stat_file=stat_file, test_key=test_key, times=stc[0].time, vertices=vertno, connectivity=connectivity, paired=True)


In [5]:
def cluster_perm_test1(X1, X2, stat_file, test_key, times=None, vertices=None, mode='a', connectivity='ico5', paired=False):
    """
    If paired, test X1-X2.
    Input: arrays of shape (subjects, time, space)
    """
    if not isinstance(X2, (np.ndarray, xr.DataArray)):
        X2 = np.zeros(X1.shape)
    
    if type(connectivity) is str:
        connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(connectivity[-1])))
    
    if paired:
        X = X1 - X2
        T_obs, clusters, cluster_pv, H0 = permutation_cluster_1samp_test(X, connectivity=connectivity, n_jobs=4)
    else:
        T_obs, clusters, cluster_pv, H0 = permutation_cluster_test([X1,X2], connectivity=connectivity, n_jobs=4)
    
    p_val = np.ones(T_obs.shape)
    clu_inds = np.zeros(T_obs.shape)
    
    for c,clu in enumerate(clusters):
        p_val[clu] = cluster_pv[c]
        clu_inds[clu] = c+1
    
    if times is not None:
        stats = np.zeros((3, *T_obs.shape))
        stats = xr.DataArray(stats, dims=['data', 'time'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'time':times})
        stats.loc['T_stat'] = T_obs
        stats.loc['p_val'] = p_val
        stats.loc['clu_inds'] = clu_inds
    
    if vertices is not None:
        stats = np.zeros((3, 2, len(vertices)))
        stats = xr.DataArray(stats, dims=['data', 'hemisphere', 'src'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'hemisphere':['left', 'right'], 'src':vertices})
        for h,hemi in enumerate(['left', 'right']):
            i = h + 1
            h *= len(vertices)
            i *= len(vertices)
            stats.loc['T_stat', hemi] = T_obs[h:i]
            stats.loc['p_val', hemi] = p_val[h:i]
            stats.loc['clu_inds', hemi] = clu_inds[h:i]
    
    os.makedirs(op.split(stat_file)[0], exist_ok=True)
    if not op.isfile(stat_file):
        mode = 'w'
    
    stats.to_netcdf(path=stat_file, group=test_key, mode=mode)
    
    return T_obs, clusters, cluster_pv, H0

In [8]:
test_key = 'RS_t_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test1(stc[1].loc['RS'].mean(dim='time').values, stc[0].loc['RS'].mean(dim='time').values, stat_file=stat_file, test_key=test_key, vertices=vertno, connectivity=connectivity, paired=True)
test_key = 'FA_t_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test1(stc[1].loc['FA'].mean(dim='time').values, stc[0].loc['FA'].mean(dim='time').values, stat_file=stat_file, test_key=test_key, vertices=vertno, connectivity=connectivity, paired=True)
test_key = 'OM_t_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = cluster_perm_test1(stc[1].loc['OM'].mean(dim='time').values, stc[0].loc['OM'].mean(dim='time').values, stat_file=stat_file, test_key=test_key, vertices=vertno, connectivity=connectivity, paired=True)


Using a threshold of 2.034515
stat_fun(H1): min=-3.472786 max=3.569783
Running initial clustering
Found 81 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.0min remaining:  2.0min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.0min finished


Computing cluster p-values
Done.
Using a threshold of 2.034515
stat_fun(H1): min=-4.245138 max=2.861377
Running initial clustering
Found 66 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.2min remaining:  2.2min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.3min finished


Computing cluster p-values
Done.
Using a threshold of 2.034515
stat_fun(H1): min=-3.930486 max=3.650433
Running initial clustering
Found 71 clusters
Permuting 1023 times...


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.0min remaining:  2.0min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  2.1min finished


Computing cluster p-values
Done.


In [ ]:
test_key = 'RS_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = mne.stats.permutation_cluster_1samp_test(stc[1].loc['RS'].mean(dim='time').values - stc[0].loc['RS'].mean(dim='time').values, connectivity=connectivity, n_jobs=4)
test_key = 'FA_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = mne.stats.permutation_cluster_1samp_test(stc[1].loc['FA'].mean(dim='time').values - stc[0].loc['FA'].mean(dim='time').values, connectivity=connectivity, n_jobs=4)
test_key = 'OM_avg'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key] = clu[test_key] = mne.stats.permutation_cluster_1samp_test(stc[1].loc['OM'].mean(dim='time').values - stc[0].loc['OM'].mean(dim='time').values, connectivity=connectivity, n_jobs=4)


In [15]:
stc[0].loc['RS'].mean(dim='time')

<xarray.DataArray (subject: 34, time: 20, src: 20484)>
array([[[ 2.008773,  2.415848, ...,  3.622698,  3.64611 ],
        [ 2.621764,  1.310435, ...,  1.530294,  1.259914],
        ...,
        [ 3.962548,  1.117905, ...,  1.488668,  1.574307],
        [ 4.025879,  1.885759, ...,  1.656659,  1.290358]],

       [[ 4.133355,  0.232459, ...,  1.052002,  0.278649],
        [ 2.540437,  0.676301, ...,  1.966223,  2.014599],
        ...,
        [ 1.747391,  1.61533 , ...,  0.507072,  0.705719],
        [ 1.883386,  1.496186, ...,  1.206225,  1.12366 ]],

       ...,

       [[10.341312,  1.209733, ..., 10.001292, 13.398304],
        [ 5.835235,  2.712691, ...,  9.20078 , 11.161971],
        ...,
        [ 3.612529,  0.464087, ...,  5.773848,  5.331923],
        [ 4.28245 ,  0.745975, ...,  5.716484,  5.330768]],

       [[ 1.972293,  3.949046, ...,  2.781159,  2.492463],
        [ 2.52077 ,  4.748508, ...,  3.543824,  3.394486],
        ...,
        [ 4.166091,  0.753249, ...,  1.060799,  

In [10]:
stop

0.245